In [9]:
from ABDB import database as db
import matplotlib.pyplot as plt
import numpy as np
from retrain_ablooper import *
import torch
import pandas as pd
from rich import print as pprint
from ABlooper import CDR_Predictor
import os

Own code for relaxing sturcutres

In [2]:
# torch settings
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_dtype(torch.float)

model = MaskDecoyGen(decoys=5).to(device = device).float()
model.load_state_dict(torch.load('best_models/best_model-2804-Radam-5-2optim', map_location=torch.device(device)))

batch_size = 1

train = torch.load('train_data/train.pt')
validation = torch.load('train_data/val.pt')
test = torch.load('train_data/test.pt')

val_dataloader = torch.utils.data.DataLoader(validation, 
                                             batch_size=batch_size,
                                             num_workers=1,
                                             shuffle=False,
                                             pin_memory=True,
                                             )

test_dataloader = torch.utils.data.DataLoader(test[:1], 
                                              batch_size=batch_size,
                                              num_workers=1,
                                              shuffle=False,
                                              pin_memory=True,
                                              )

In [3]:
for i in range(5):
    tmp = MaskDecoyGen(decoys=1).to(device = device).float()
    dict=torch.load("best_models/best_model-0305-Radam-1-2optim-"+str(i+1), map_location=torch.device(device))
    tmp.load_state_dict(dict)
    weights = tmp.blocks[0].state_dict()

    model.blocks[i].load_state_dict(weights)

In [4]:
def produce_full_structures_of_val_set(val_dataloader, model, outdir='', relax=True, to_be_rewritten=["H1", "H2", "H3", "L1", "L2", "L3"]):
    '''
    Produces full FAB structure for a dataset
    '''
    CDR_rmsds_not_relaxed = list()
    CDR_rmsds_relaxed = list()
    decoy_diversities = list()
    order_of_pdbs = list()

    with torch.no_grad():
        model.eval()

        for data in track(val_dataloader, description='predict val set'):

            # predict sturcture using the model
            coordinates, geomout, node_feature, mask, id = data['geomins'].float().to(device), data['geomouts'].float().to(device), data['encodings'].float().to(device), data['mask'].float().to(device), data['ids']
            pred = model(node_feature, coordinates, mask)
            CDR_rmsds_not_relaxed.append(rmsd_per_cdr(pred, node_feature, geomout).tolist())
            pred = pred.squeeze() # remove batch dimension
            
            # get framework info from pdb file
            pdb_id, heavy_c, light_c, pdb_file = get_info_from_id(id)
            chains = [heavy_c, light_c]
            order_of_pdbs.append(pdb_id)

            with open(pdb_file) as file:
                pdb_text = [line for line in file.readlines()]
                
            pdb_text = pdb_select_hc_lc(pdb_text, chains)

            CDR_with_anchor_slices, atoms, CDR_text, CDR_sequences, CDR_numberings, CDR_start_atom_id = get_framework_info(pdb_text, chains)

            predicted_CDRs = {}
            all_decoys = {}
            decoy_diversity = {}

            for i, CDR in enumerate(CDR_with_anchor_slices):
                output_CDR = pred[:, node_feature[0, :, 30 + i] == 1.0]
                all_decoys[CDR] = rearrange(output_CDR, "b (i a) d -> b i a d", a=4).cpu().numpy()
                predicted_CDRs[CDR] = rearrange(output_CDR.mean(0), "(i a) d -> i a d", a=4).cpu().numpy()
                decoy_diversity[CDR] = (output_CDR[None] - output_CDR[:, None]).pow(2).sum(-1).mean(-1).pow(
                    1 / 2).sum().item() / 20
            
            decoy_diversities.append(list(decoy_diversity.values()))
            
            text_prediction_per_CDR = convert_predictions_into_text_for_each_CDR(CDR_start_atom_id, predicted_CDRs, CDR_sequences, CDR_numberings, CDR_with_anchor_slices)
            old_text = pdb_text

            for CDR in to_be_rewritten:
                new = True
                new_text = []
                chain, CDR_slice = CDR_with_anchor_slices[CDR]
                CDR_slice = (CDR_slice[0] + 2, CDR_slice[1] - 2)

                for line in old_text:
                    if not filt(line, chain, CDR_slice):
                        new_text.append(line)
                    elif new:
                        new_text += text_prediction_per_CDR[CDR]
                        new = False
                    else:
                        continue
                old_text = new_text

            header = [
                "REMARK    CDR LOOPS REMODELLED USING ABLOOPER                                   \n"]
            new_text = header + old_text

            with open('pdbs/'+outdir+'/'+pdb_id+'-'+heavy_c+light_c+'.pdb', "w+") as file:
                file.write("".join(new_text))

            with open('pdbs/'+outdir+'/'+pdb_id+'-'+heavy_c+light_c+'-true.pdb', "w+") as file:
                file.write("".join(pdb_text))

            if relax:
                relaxed_text = openmm_refine(old_text, CDR_with_anchor_slices)
                header.append("REMARK    REFINEMENT DONE USING OPENMM" + 42 * " " + "\n")
                relaxed_text = header + relaxed_text

                with open('pdbs/'+outdir+'/'+pdb_id+'-'+heavy_c+light_c+'-relaxed.pdb', "w+") as file:
                    file.write(''.join(relaxed_text))


                # calculate rmsds of relaxed structures
                CDR_with_anchor_slices, atoms, CDR_text, CDR_sequences, CDR_numberings, CDR_start_atom_id = get_framework_info(relaxed_text, chains)
                
                CDR_BB_coords = extract_BB_coords(CDR_text, CDR_with_anchor_slices, CDR_sequences, atoms)

                relaxed_coords = prepare_model_output([CDR_BB_coords])[0]
            
                relaxed_coords = pad_tensor(relaxed_coords)
                relaxed_coords = rearrange(relaxed_coords, 'i x -> () () i x')
                CDR_rmsds_relaxed.append(rmsd_per_cdr(relaxed_coords, node_feature, geomout).tolist())


    return CDR_rmsds_not_relaxed, CDR_rmsds_relaxed, decoy_diversities, order_of_pdbs

In [14]:
def cdr_rmsd_of_2_pdb_files(prediction_path, true_path, chains):
    
    with open(prediction_path, 'r') as file:
        pdb_text_pred = [line for line in file.readlines()]

    with open(true_path, 'r') as file:
        pdb_text_true = [line for line in file.readlines()]

    pdb_text_true = pdb_select_hc_lc(pdb_text_true, chains)
    pdb_text_pred = pdb_select_hc_lc(pdb_text_pred, chains)
    
    CDR_with_anchor_slices, atoms, CDR_text_true, _, _, _ = get_framework_info(pdb_text_true, chains)
    _, _, CDR_text_pred, _, _, _ = get_framework_info(pdb_text_pred, chains)
    
    CDR_texts = [CDR_text_true, CDR_text_pred]
    for CDR_text in CDR_texts:
        for CDR in CDR_with_anchor_slices:
            new = [line for line in CDR_text[CDR] if line.split()[2] in atoms]
            new = [line for line in new if (line.split()[5][-1].isalpha() or int(line.split()[5]) >= CDR_with_anchor_slices[CDR][1][0] + 2)]
            new = [line for line in new if (line.split()[5][-1].isalpha() or int(line.split()[5]) <= CDR_with_anchor_slices[CDR][1][1] - 2)]
        
            CDR_text[CDR] = new
    
    CDR_BB_coors_true = {}
    CDR_BB_coors_pred = {}
    CDR_BB_coors = [CDR_BB_coors_true, CDR_BB_coors_pred]
    for i in range(len(CDR_texts)):
        for CDR in CDR_with_anchor_slices:
            coords = [[float(line.split()[6]), float(line.split()[7]), float(line.split()[8])] for line in CDR_texts[i][CDR]]
            coords = np.array(coords)
            
            CDR_BB_coors[i][CDR] = coords
            
    rmsds = []
    for CDR in CDR_with_anchor_slices:
        true = CDR_BB_coors[0][CDR]
        pred = CDR_BB_coors[1][CDR]
        
        rmsd = np.sqrt(((pred - true) ** 2).sum(-1).mean())
        rmsds.append(rmsd)
        
        
    return np.array(rmsds)

In [37]:
dir = '/data/localhost/not-backed-up/spoendli/pdbs/0305-Radam-1-2optim-test/'

files = os.listdir(dir)
files = [file for file in files if file != 'metrics.json']

files = [file[:7] for file in files if file[0] != '.']
files = list(set(files))

ids = []
rmsds = []

for i in range(len(files)):
    chains = (files[i][-2], files[i][-1])
        
    true_path = dir+files[i]+'-true.pdb'
    relaxed_path = dir+files[i]+'-relaxed.pdb'
    
    ids.append(files[i])
    rmsds.append(cdr_rmsd_of_2_pdb_files(relaxed_path, true_path, chains))

rmsds = np.array(rmsds)

In [38]:
rmsds.mean(0)

array([1.17081381, 1.09165272, 2.46593909, 0.90204051, 0.92089803,
       1.13564884])

In [31]:
ids

['2r8s-HL',
 '4nzu-HL',
 '3oz9-HL',
 '1dlf-HL',
 '3p0y-HL',
 '3mlr-HL',
 '2d7t-HL',
 '2fbj-HL',
 '1fns-HL',
 '2ypv-HL',
 '1mqk-HL',
 '2w60-AB',
 '3lmj-HL',
 '2adf-HL',
 '2e27-HL',
 '1nlb-HL',
 '3m8o-HL',
 '3nps-BC',
 '4f57-HL',
 '4hpy-HL',
 '1jpt-HL',
 '3e8u-HL',
 '1mlb-BA',
 '2fb4-HL',
 '3liz-HL',
 '2v17-HL',
 '3hnt-HL',
 '3go1-HL',
 '1mfa-HL',
 '3hc4-HL',
 '4h20-HL',
 '1oaq-HL',
 '3giz-HL',
 '1gig-HL',
 '3mxw-HL',
 '3gnm-HL',
 '2vxv-HL',
 '2xwt-AB',
 '3ifl-HL',
 '3g5y-BA',
 '3i9g-HL',
 '3v0w-HL',
 '1seq-HL',
 '3eo9-HL',
 '3t65-BA',
 '1jfq-HL']

In [311]:
a = np.ones((5,3))
a[:,0] = 10
b = np.zeros((5,3))

c = np.sqrt(((a - b) ** 2).sum(-1).mean())
c

10.099504938362077

In [ ]:
def rmsd(prediction, truth):
    dists = (prediction - truth).pow(2).sum(-1)
    return torch.sqrt(dists.nanmean(-1)).mean()

In [119]:
coors = {}
for CDR in original:
    
    original[CDR]
    
    coors[CDR] = 
    

SyntaxError: invalid syntax (1984536977.py, line 6)

In [126]:
dir = '0305-Radam-1-2optim-test'

In [127]:
cdr_rmsds, CDR_rmsds_relaxed, decoy_diversities, pdb_ids = produce_full_structures_of_val_set(test_dataloader, 
                                                                                              model, 
                                                                                              outdir=dir, 
                                                                                              relax=True)

/data/localhost/not-backed-up/spoendli/miniconda3/envs/ab/lib/python3.8/site-packages/rich/li
ve.py:221: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [135]:
pdb_ids, cdr_rmsds, CDR_rmsds_relaxed

(['3oz9'],
 [[0.39621397852897644,
   0.5520344972610474,
   2.0543265342712402,
   0.5045629143714905,
   0.1996065378189087,
   1.3235737085342407]],
 [[1.6838728189468384,
   1.7694529294967651,
   2.6591641902923584,
   1.8018285036087036,
   1.6651703119277954,
   2.0778539180755615]])

In [55]:
l = [x for x in original['H1'] if x.split()[2] in ['N', 'CA', 'CB', 'C']]
l

['ATOM    171  N   ALA H  25      60.477  28.466  54.002  1.00 13.07           N  \n',
 'ATOM    172  CA  ALA H  25      60.948  28.433  55.380  1.00 15.14           C  \n',
 'ATOM    173  C   ALA H  25      62.231  29.226  55.523  1.00 17.82           C  \n',
 'ATOM    175  CB  ALA H  25      61.180  26.997  55.824  1.00 16.47           C  \n',
 'ATOM    176  N   THR H  26      62.279  30.078  56.536  1.00 14.49           N  \n',
 'ATOM    177  CA  THR H  26      63.498  30.821  56.826  1.00 18.07           C  \n',
 'ATOM    178  C   THR H  26      63.792  30.784  58.324  1.00 18.00           C  \n',
 'ATOM    180  CB  THR H  26      63.404  32.287  56.333  1.00 16.08           C  \n',
 'ATOM    183  N   GLY H  27      65.053  31.016  58.682  1.00 17.47           N  \n',
 'ATOM    184  CA  GLY H  27      65.443  31.146  60.079  1.00 17.45           C  \n',
 'ATOM    185  C   GLY H  27      65.899  29.883  60.788  1.00 20.90           C  \n',
 'ATOM    187  N   TYR H  28      65.988  2

In [16]:
relaxed['H1']

['ATOM    354  N   ALA H  25      60.517  28.515  54.001  1.00  0.00           N  \n',
 'ATOM    355  H   ALA H  25      60.953  27.888  53.334  1.00  0.00           H  \n',
 'ATOM    356  CA  ALA H  25      61.026  28.506  55.366  1.00  0.00           C  \n',
 'ATOM    357  HA  ALA H  25      60.261  28.914  56.026  1.00  0.00           H  \n',
 'ATOM    358  C   ALA H  25      62.311  29.338  55.550  1.00  0.00           C  \n',
 'ATOM    359  O   ALA H  25      63.269  29.250  54.776  1.00  0.00           O  \n',
 'ATOM    360  CB  ALA H  25      61.246  27.055  55.783  1.00  0.00           C  \n',
 'ATOM    361  HB1 ALA H  25      61.553  27.013  56.829  1.00  0.00           H  \n',
 'ATOM    362  HB2 ALA H  25      62.032  26.639  55.156  1.00  0.00           H  \n',
 'ATOM    363  HB3 ALA H  25      60.326  26.482  55.651  1.00  0.00           H  \n',
 'ATOM    364  N   THR H  26      62.355  30.075  56.658  1.00  0.00           N  \n',
 'ATOM    365  H   THR H  26      61.489  3

In [115]:
geomout[0,:273,0]

tensor([60.9480, 62.2310, 60.4770, 61.1800, 63.4980, 63.7920, 62.2790, 63.4040,
        65.4430, 65.8990, 65.0530, 65.4430, 66.4170, 66.8900, 65.9880, 65.2770,
        67.9720, 66.8080, 67.4860, 68.9610, 65.3310, 65.1680, 66.5260, 65.3590,
        66.1990, 66.0500, 66.2730, 67.4310, 66.0760, 64.6810, 66.0780, 66.7170,
        62.3010, 61.3060, 63.6550, 61.8780, 59.1770, 58.3190, 60.2290, 58.3120,
        56.9400, 55.5710, 57.9280, 57.2240, 53.3530, 52.3910, 54.6930, 52.8950,
        50.4880, 51.7270, 49.2830, 50.4880, 54.1420, 55.2470, 52.8860, 54.5760,
        57.4170, 58.6890, 56.2460, 57.3710, 61.0100, 61.9620, 59.7240, 61.5860,
        63.1300, 64.5160, 62.2700, 63.2430, 66.4050, 66.1950, 65.1150, 66.4050,
        65.6810, 64.3920, 65.9960, 65.5360, 62.1910, 61.1460, 63.4380, 62.1910,
        60.3810, 59.0370, 61.3370, 60.8250, 56.6260, 55.7500, 57.9670, 55.9540,
        53.7410, 52.4030, 54.7760, 54.0310, 50.1010, 48.9970, 51.4210, 49.7300,
        48.5000, 49.6600, 47.3530, 48.16

In [116]:
relaxed_coords[:273,0]

tensor([61.0140, 60.5060, 62.2730, 61.2790, 63.5370, 62.3400, 63.4960, 63.5940,
        65.9160, 64.5120, 66.3640, 65.9160, 66.4880, 66.2080, 65.9540, 65.8370,
        67.8880, 66.7950, 67.2600, 68.9720, 65.5920, 66.2430, 66.5310, 65.0230,
        67.0880, 66.8620, 66.5880, 68.5410, 66.2960, 67.0550, 65.0950, 67.1920,
        62.6780, 64.0420, 61.8650, 62.5410, 59.6660, 60.7960, 58.9410, 58.7090,
        57.1050, 58.1740, 55.8000, 57.3500, 53.5220, 54.8010, 52.4130, 53.3290,
        50.4890, 49.2750, 51.7410, 50.4890, 54.2160, 52.9160, 55.2890, 54.6680,
        57.4890, 56.3510, 58.6690, 57.3530, 61.0920, 59.7980, 61.8400, 61.7990,
        63.5260, 62.6140, 63.7810, 63.0470, 65.5560, 64.7830, 66.6890, 65.5560,
        65.5380, 66.5590, 64.1090, 65.8070, 62.1730, 63.4790, 61.0310, 62.1730,
        60.4440, 61.2700, 59.1330, 61.1080, 56.7310, 58.0430, 55.8290, 56.2610,
        53.7740, 54.8350, 52.4420, 54.0860, 50.0960, 51.4050, 49.0030, 49.7890,
        48.5100, 47.3390, 49.6570, 48.20

In [62]:
rearrange(relaxed_coords, 'a b -> () () a b')

tensor([[[[61.0140, 28.4920, 55.3740],
          [60.5060, 28.5050, 54.0070],
          [62.2730, 29.3600, 55.5760],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]]]])

In [49]:
rmsd(relaxed_coords, geomout)

tensor(1.5104)

In [13]:
cdr_rmsds

[[0.39621397852897644,
  0.5520344972610474,
  2.0543265342712402,
  0.5045629143714905,
  0.1996065378189087,
  1.3235737085342407],
 [1.4963270425796509,
  1.0473147630691528,
  2.906479835510254,
  0.6791009902954102,
  0.2234359234571457,
  1.0305887460708618]]

In [14]:
 CDR_rmsds_relaxed

[[0.39689359068870544,
  0.5317484736442566,
  2.0721490383148193,
  0.5045245885848999,
  0.19938620924949646,
  1.3232002258300781],
 [1.4967271089553833,
  1.0466426610946655,
  2.9059810638427734,
  0.6716121435165405,
  0.2233588546514511,
  1.0306532382965088]]

Calculate rmsds of relaxed structures

In [66]:
import os
from ABDB import database as db
import matplotlib.pyplot as plt
import numpy as np
from retrain_ablooper import *
import torch
import pandas as pd
from rich import print as pprint
from ABlooper import CDR_Predictor
from einops import rearrange

In [86]:
dir = '/data/localhost/not-backed-up/spoendli/pdbs/0305-Radam-1-2optim-test/'

In [104]:
files = os.listdir(dir)
[file for file in files if file != 'metrics.json']

['3i9g-HL.pdb',
 '4nzu-HL.pdb',
 '4nzu-HL-true.pdb',
 '3go1-HL.pdb',
 '4hpy-HL-true.pdb',
 '1gig-HL-relaxed.pdb',
 '3g5y-BA-true.pdb',
 '3giz-HL-relaxed.pdb',
 '2fb4-HL.pdb',
 '1mlb-BA.pdb',
 '3gnm-HL-relaxed.pdb',
 '1dlf-HL-true.pdb',
 '2fb4-HL-relaxed.pdb',
 '3m8o-HL-true.pdb',
 '2e27-HL-true.pdb',
 '1gig-HL.pdb',
 '3t65-BA-true.pdb',
 '2ypv-HL-true.pdb',
 '1mfa-HL-true.pdb',
 '3go1-HL-relaxed.pdb',
 '1mfa-HL.pdb',
 '2xwt-AB-true.pdb',
 '2fbj-HL-relaxed.pdb',
 '4f57-HL-true.pdb',
 '3g5y-BA-relaxed.pdb',
 '3v0w-HL-relaxed.pdb',
 '2vxv-HL-relaxed.pdb',
 '3liz-HL.pdb',
 '3i9g-HL-relaxed.pdb',
 '1mfa-HL-relaxed.pdb',
 '3ifl-HL-relaxed.pdb',
 '3p0y-HL.pdb',
 '3eo9-HL-true.pdb',
 '2d7t-HL.pdb',
 '2e27-HL-relaxed.pdb',
 '3t65-BA-relaxed.pdb',
 '3ifl-HL-true.pdb',
 '2d7t-HL-true.pdb',
 '3m8o-HL.pdb',
 '3mlr-HL-true.pdb',
 '4h20-HL-true.pdb',
 '2r8s-HL.pdb',
 '3mxw-HL.pdb',
 '3oz9-HL-true.pdb',
 '3go1-HL-true.pdb',
 '3t65-BA.pdb',
 '2vxv-HL-true.pdb',
 '2r8s-HL-true.pdb',
 '2w60-AB-true.pdb',

In [109]:
def cdr_rmsds_on_pdb_file(prediction_path, truth_path, chains):

    pred = CDR_Predictor(prediction_path, chains = chains)
    pred_coords = prepare_model_output([pred.CDR_BB_coords])[0]
    pred_coords = rearrange(pred_coords, 'i x -> () () i x')
    pred_nodefeatures = pred.prepare_model_input()[0]
    
    truth = CDR_Predictor(truth_path, chains)
    truth_coords = prepare_model_output([truth.CDR_BB_coords])[0]
    truth_coords = rearrange(truth_coords, 'i x -> () i x')

    cdr_rmsds = rmsd_per_cdr(pred_coords, pred_nodefeatures, truth_coords)

    return cdr_rmsds

def rmsds_pred_and_relaxed(dir):

    files = os.listdir(dir)
    files.remove('metrics.json')
    files = [file[:7] for file in files if file[0] != '.']
    files = list(set(files))
    
    ids = list()
    rmsds_pred = np.zeros((len(files), 6))
    rmsds_relaxed = np.zeros((len(files), 6))

    for i in range(len(files)):
        chains = (files[i][-2], files[i][-1])
        
        truth_path = dir+files[i]+'-true.pdb'
        pred_path = dir+files[i]+'.pdb'
        relaxed_path = dir+files[i]+'-relaxed.pdb'

        ids.append(files[i])
        rmsds_pred[i,:] = np.array(cdr_rmsds_on_pdb_file(pred_path, truth_path, chains).tolist())
        rmsds_relaxed[i,:] = np.array(cdr_rmsds_on_pdb_file(relaxed_path, truth_path, chains).tolist())

    return ids, rmsds_pred, rmsds_relaxed

In [110]:
ids, rmsd_pred, rmsd_relaxed = rmsds_pred_and_relaxed(dir)


In [114]:
rmsd_relaxed.mean(0)

array([1.16327669, 1.09427714, 2.47645663, 0.90324465, 0.92075492,
       1.13739726])

In [117]:
from ABDB import database as db
from ABDB.AbPDB import AntibodyParser
import numpy as np
import Bio.PDB
parser = AntibodyParser(PERMISSIVE=True, QUIET=True)
parser.set_numbering_scheme("imgt")
db.set_numbering_scheme("imgt")
backbone = ["CA","C","N", "CB"]

def CDR_rmsds(pdb, file, fab_n = 0, chains = ["H", "L"], decoy_chains = ["H", "L"]):
    fab = db.fetch(pdb).fabs[fab_n]
    rmsds = {}
    for h_or_l in chains:
        #Truth load
        chain = db.db_summary[pdb]["fabs"][fab_n][h_or_l+"chain"]
        truth = fab.get_structure()[chain]
        #Decoy load
        decoy = parser.get_antibody_structure(pdb+"_model", file)[0][{"H":decoy_chains[0], "L":decoy_chains[1]}[h_or_l]]
        #Numbering
        numb = [(" ", *x) for x in fab.get_numbering()[h_or_l]][2:-2]
        #Get residues to align
        truth_res = [truth[x] for x in numb if (x in decoy) and (x in truth)]
        decoy_res = [decoy[x] for x in numb if (x in decoy) and (x in truth)]
        #Get atoms to align
        fixed = []
        moved = []
        for i in range(len(decoy_res)):
            fixed += [truth_res[i][atom] for atom in backbone if (atom in decoy_res[i]) and (atom in truth_res[i])]
            moved += [decoy_res[i][atom] for atom in backbone if (atom in decoy_res[i]) and (atom in truth_res[i])]
        #Calculate superimposer and move decoy
        imposer = Bio.PDB.Superimposer()
        imposer.set_atoms(fixed, moved)
        imposer.apply(decoy.get_atoms())
        rmsds[h_or_l] = imposer.rms
        # Find CDR definitions
        loop_definitions = {x[3:]:[(" ", *y[0]) for y in fab.get_CDR_sequences(definition="imgt")[x]]  for x in fab.get_CDR_sequences(definition="imgt")}
        # Calculate RMSD for each CDR
        for CDR in loop_definitions:
            if CDR[0] == h_or_l:
                true_loop = []
                decoy_loop = []
                for res in loop_definitions[CDR]:
                    if (res in truth) and (res in decoy):
                        true_loop += [truth[res][x].get_coord() for x in backbone if (x in decoy[res]) and (x in truth[res])]
                        decoy_loop+= [decoy[res][x].get_coord() for x in backbone if (x in decoy[res]) and (x in truth[res])]
                #Calculate RMSD
                rmsds[CDR] = np.sqrt(np.mean(3*(np.array(true_loop) - np.array(decoy_loop))**2))
                
    # Calculate RMSD for framework
    ignore = sum([loop_definitions[x] for x in loop_definitions if x[0] == h_or_l], [])
    frame_def = [x for x in numb if x not in ignore]
    
    true_frame, decoy_frame = [], []
    for res in frame_def:
        if (res in truth) and (res in decoy):
            true_frame += [truth[res][x].get_coord() for x in backbone if (x in decoy[res]) and (x in truth[res])]
            decoy_frame+= [decoy[res][x].get_coord() for x in backbone if (x in decoy[res]) and (x in truth[res])]
    rmsds[h_or_l] = np.sqrt(np.mean(3*(np.array(true_frame) - np.array(decoy_frame))**2))
                
    return rmsds

In [137]:
pdb = db.fetch('5hdq')
for fab in pdb.fabs:
    print(fab)

fab HL
	VH = chain H
	VL = chain L


In [139]:
CDR_rmsds('3oz9', '/data/localhost/not-backed-up/spoendli/pdbs/0305-Radam-1-2optim-test/3oz9-HL.pdb', fab_n = 0, chains = ["H", "L"], decoy_chains = ["H", "L"])

{'H': 1.0492338462729975,
 'H1': 1.7875236,
 'H2': 1.8640847,
 'H3': 2.464813,
 'L': 0.072361894,
 'L1': 1.7554567,
 'L2': 1.762282,
 'L3': 2.0607486}